## Plot results for panels project
- model-predicted panel labels
- NEC calculations for existing loads

In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import random

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


In [ ]:
model = "41138" # <--- "162", "16570", "90298", "41146", "16566", "41138", "41906", "35006", "90290"

main_dir = Path("C:/Users/ylou2/Desktop/panel_size_prediction/")
# following two files are generated by LBNL's DecisionTree model
model_file = main_dir / f"euss1_2018_results_up00_clean__model_{model}__tsv_based__predicted_panels_probablistically_assigned.csv"
model_file_prob = main_dir / f"euss1_2018_results_up00_clean__model_{model}__tsv_based__predicted_panels_in_probability.csv"
# following one file is generated by postprocess_panel_existing_load_nec.py
nec_file = main_dir / "euss1_2018_results_up00_clean__existing_load.csv"
model_training_file = Path(f"C:/Users/ylou2/Desktop/resstock/Electrical_Panels_EUSS/panels/model_20240327/model_data_{model}.xlsx")

### Specify filters to apply to all data

In [ ]:
### specify filters as query
has_filter = False # <---

if has_filter:
    filter_query = "`build_existing_model.state` == 'CA' & `build_existing_model.geometry_building_type_recs` == 'Single-Family Detached'" # <---
    folder_ext = "__CA_SFD_only" # <---
    print(f"All input datasets will be filtered using query statement: \n{filter_query}")
else:
    folder_ext = ""
    print("No filter will be applied to the input datasets")

In [ ]:

output_dir = main_dir / "plots" / f"model_{model}_for_paper{folder_ext}"
output_dir.mkdir(exist_ok=True, parents=True)
print(f"All results will be output to: \n{output_dir}")

### Transform binned output into weighted representative values

In [ ]:
def undummify_input_data(df, prefix_sep="__"):
    # undummy categorical columns
    cat_cols = [x for x in df.columns if prefix_sep in x]
    other_cols = [x for x in df.columns if x not in cat_cols]
    dfd = undummify(df[cat_cols], prefix_sep=prefix_sep)

    # bin sqft
    para = "geometry_floor_area"
    bins = ["0-499", "500-749", "750-999", "1000-1499", "1500-1999", "2000-2499", "2500-2999", "3000-3999", "4000+"]
    bin_edges = [int(x.split("-")[0].split("+")[0]) for x in bins]
    dfd.loc[df["sqft"]>=bin_edges[-1], para] = bins[-1]
    for edge, label in zip(reversed(bin_edges[1:]), reversed(bins[:-1])):
        dfd.loc[df["sqft"]<edge, para] = label

    dfd = pd.concat([dfd, df[other_cols]], axis=1)

    return dfd


def undummify(df, prefix_sep="__"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)

    return undummified_df


def save_filtered_df_to_file(df, original_filename: Path, output_dir: Path | None):
    new_file = original_filename.stem + "__filtered.csv"
    if output_dir is None:
        new_filename = original_filename.parent / new_file
    else:
        new_filename = output_dir / new_file

    df.to_csv(new_filename, index=False)



In [ ]:
# Load training data and undummify:
dfd = pd.read_excel(model_training_file, sheet_name=f"input_{model}", header=0)
output_mapping = {
            0: "100",
            1: "101-199",
            2: "200",
            3: "201+",
            4: "<100", # "lt_100",
        }
dfd["panel_amp_pre_bin"] = dfd["panel_amp_pre_bin"].map(output_mapping)
dfd = undummify_input_data(dfd)
dfd

### Pick out standard sizes for each bin's representative numbers:
value (top # place mode):
- <100: 90(1st), 70(2nd), 60(3rd), 30(4th)
- 101-199: 125(1st), 150(2nd), 175(4th) # note: 120(3rd) and 110(5th) will be combined with 125
- 201+: [225, 250, 300, 350, 400, 500, ..., 1000], based on top 5 modes: [250, 300, 225, 400, 220]

Using this to visualize:\
`dfd.loc[(dfd["panel_amp_pre"]<1000) & (dfd["panel_amp_pre_bin"]=="<100"), "panel_amp_pre"].value_counts().sort_index()`\
`dfd.loc[dfd["panel_amp_pre"]<10000,["panel_amp_pre_bin", "panel_amp_pre"]].boxplot(by="panel_amp_pre_bin");
plt.show()`


In [ ]:
# standardize panel_amp_pre values by rounding to the nearest bin defined above
standard_sizes = {
    "<100": np.array([30, 60, 70, 90]),
    "100": np.array([100]),
    "101-199": np.array([125, 150, 175]),
    "200": np.array([200]),
    "201+": np.concatenate([
        np.array([225, 250, 300, 350]), 
        np.arange(400, 1100, 100)
    ])
}
for lab, std_sizes in standard_sizes.items():
    cond = dfd["panel_amp_pre_bin"]==lab
    dfd.loc[cond, "panel_amp_pre_std"] = dfd.loc[cond, "panel_amp_pre"].apply(lambda x: std_sizes[np.argmin(np.abs(std_sizes/x-1))])

dfd


In [ ]:
# turn standardized panel amps into rep value distribution
gb = ["panel_amp_pre_bin",]
panel_value_dist = (dfd.groupby(gb+["panel_amp_pre_std"])["sqft"].count() / dfd.groupby(gb)["sqft"].count()
                   ).rename("prob").reset_index(level='panel_amp_pre_std')
panel_value_dist = panel_value_dist.groupby(level=gb).agg({'panel_amp_pre_std': list, 'prob': list}).agg(tuple, axis=1).rename("panel_amp_dist")
panel_value_dist

In [ ]:
def sample_panel_size_unweighted(x):
    """ sample panel size with unweighted (i.e., equally weighted) """
    if x == "<100":
        labels = np.array([30, 60, 70, 90])
        weights = np.ones(len(labels))
    elif x == "101-199":
        labels = np.array([125, 150, 175])
        weights = np.ones(len(labels))
    elif x == "201+":
        labels = np.concatenate([
            np.array([225, 250, 300, 350]), 
            np.arange(400, 1100, 100)
        ])
        weights = np.ones(len(labels))
    else:
        return int(x)
    return random.choices(labels, weights=weights, k=1)[0]

# for use with rep value distribution
def weighted_sample(x):
    return random.choices(x[0], weights=x[1], k=1)[0]
    

In [ ]:
### rep value lookup derived directly from data is all over the place... so we probably need to create this manually...
dfd_lkup = dfd.copy()
dfd_lkup["building_type"] = dfd_lkup["geometry_building_type_recs_simp"].map({
    "Mobile Home": "SFD/MH", 
    "Multi-Family with 2 - 4 Units": "MF",
    "Multi-Family with 5+ Units": "MF", 
    "Single-Family Attached": "SF",
    "Single-Family Detached": "SF", 
})
dfd_lkup.groupby(["building_type", "panel_amp_pre_bin"])["panel_amp_pre"].mean()

In [ ]:
dfd_lkup.loc[dfd_lkup["geometry_floor_area"].isin(["0-499", "500-749"])]\
.groupby(["geometry_floor_area", "panel_amp_pre_bin"])["panel_amp_pre"].describe() #.agg(["count", "mean"])

In [ ]:
dfd_lkup.loc[(dfd_lkup["geometry_floor_area"].isin(["500-749"])) & (dfd_lkup["panel_amp_pre_bin"]=="201+"), 
["geometry_floor_area", "geometry_building_type_recs_simp", "panel_amp_pre_bin", "panel_amp_pre"]]

In [ ]:
### predicted panel sizes
method = "weighted" # <--- unweighted, weighted, lookup (still need to be constructed)

if has_filter:
    df = pd.read_csv(model_file, low_memory=False, keep_default_na=True).query(filter_query).reset_index(drop=True)
    save_filtered_df_to_file(df, model_file, output_dir=output_dir)
else:
    df = pd.read_csv(model_file, low_memory=False, keep_default_na=False)
df["predicted_panel_amp"] = df["predicted_panel_amp"].astype(str)

# sample exact values for panel labels
random.seed(10)
if method == "unweighted":
    df["predicted_panel_amp_exact"] = df["predicted_panel_amp"].apply(sample_panel_size_unweighted).astype(int)
elif method == "weighted":
    df["predicted_panel_amp_exact"] = df["predicted_panel_amp"].map(panel_value_dist).apply(weighted_sample).astype(int)
else:
    raise NotImplementedError(f"{method=} not implemented")

categories = ["<100", "100", "101-199", "200", "201+"]
df["predicted_panel_amp"] = pd.Categorical(df["predicted_panel_amp"], ordered=True, categories=categories)

if has_filter:
    dfm = pd.read_csv(model_file_prob, low_memory=False, keep_default_na=True).query(filter_query).reset_index(drop=True)
    save_filtered_df_to_file(dfm, model_file_prob, output_dir=output_dir)
else:
    dfm = pd.read_csv(model_file_prob, low_memory=False, keep_default_na=False)
dfm = pd.concat([dfm, df[["predicted_panel_amp", "predicted_panel_amp_exact"]]], axis=1)
del df

# occupied only
dfm = dfm.loc[dfm["build_existing_model.vacancy_status"]=="Occupied"].reset_index(drop=True)
dfm

In [ ]:
### NEC calc of existing loads
if has_filter:
    dfn = pd.read_csv(nec_file, low_memory=False, keep_default_na=True).query(filter_query).reset_index(drop=True)
    save_filtered_df_to_file(dfn, nec_file, output_dir=output_dir)
else:
    dfn = pd.read_csv(nec_file, low_memory=False, keep_default_na=False)

# occupied only
dfn = dfn.loc[dfn["build_existing_model.vacancy_status"]=="Occupied"].reset_index(drop=True)
dfn["existing_amp_220_83"] = dfn["existing_amp_220_83"].astype(float)
dfn["existing_amp_220_87"] = dfn["existing_amp_220_87"].astype(float)
dfn

### plot funcs

In [ ]:
def extract_left_edge(val):
    # for sorting things like AMI
    if val is None:
        return np.nan
    if not isinstance(val, str):
        return val
    first = val[0]
    if re.search(r"\d", val) or first in ["<", ">"] or first.isdigit():
        vals = [int(x) for x in re.split("\-| |\%|\<|\+|\>|s|th|p|A|B|C| ", val) if re.match("\d", x)]
        if len(vals) > 0:
            num = vals[0]
            if "<" in val:
                num -= 1
            if ">" in val:
                num += 1
            return num
    return val
    
def sort_index(df, axis="index", **kwargs):
    """ axis: ['index', 'columns'] """
    if axis in [0, "index"]:
        try:
            df = df.reindex(sorted(df.index, key=extract_left_edge, **kwargs))
        except TypeError:
            df = df.reindex(sorted(df.index, **kwargs))
        return df

    if axis in [1, "columns"]:
        col_index_name = df.columns.name
        try:
            cols = sorted(df.columns, key=extract_left_edge, **kwargs)
        except TypeError:
            cols = sorted(df.columns, **kwargs)
        df = df[cols]
        df.columns.name = col_index_name
        return df
    raise ValueError(f"axis={axis} is invalid")

def format_labels(lst):
    flst = []
    for idx in lst:
        if len(idx)/15 > 1:
            size = math.ceil(len(idx) / 2)
            part1 = idx[:size]
            part2 = idx[size:]
            
            for i in range(1, len(part1)+1):
                if part1[-i] == " ":
                    break
            
            for j in range(len(part2)):
                if part2[j] == " ":
                    break
                    
            if i > j:
                k = len(part1)+j
            else:
                k = len(part1)-i
            
            flst.append(idx[:k]+"\n"+idx[k+1:])
        else:
            flst.append(idx)
    return flst

### plots

In [ ]:
### make categories for plotting
dfm["build_existing_model.vintage"] = pd.Categorical(dfm["build_existing_model.vintage"], ordered=True, 
        categories=['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s','2010s'])
dfm["build_existing_model.vintage_acs"] = pd.Categorical(dfm["build_existing_model.vintage_acs"], ordered=True, 
        categories=['<1940', '1940-59', '1960-79', '1980-99', '2000-09', '2010s'])
dfm["build_existing_model.geometry_floor_area"] = pd.Categorical(dfm["build_existing_model.geometry_floor_area"], ordered=True, 
        categories=['0-499', '500-749', '750-999', '1000-1499', '1500-1999', '2000-2499', '2500-2999', '3000-3999', '4000+'])
dfm["build_existing_model.geometry_floor_area_bin"] = pd.Categorical(dfm["build_existing_model.geometry_floor_area_bin"], ordered=True, 
        categories=['0-1499', '1500-2499', '2500-3999', '4000+'])
dfm["build_existing_model.federal_poverty_level"] = pd.Categorical(dfm["build_existing_model.federal_poverty_level"], ordered=True, 
        categories=['0-100%', '100-150%', '150-200%', '200-300%', '300-400%', '400%+'])
dfm["build_existing_model.area_median_income"] = pd.Categorical(dfm["build_existing_model.area_median_income"], ordered=True, 
        categories=['0-30%', '30-60%', '60-80%', '80-100%', '100-120%', '120-150%', '150%+'])

### [1] plot probability of labels by one variable

In [ ]:
groupby_cols = [
    "build_existing_model.state",
    "build_existing_model.vintage",
    "build_existing_model.geometry_floor_area",
    "build_existing_model.geometry_building_type_recs",
    "build_existing_model.census_region", 
    "build_existing_model.federal_poverty_level",
    "build_existing_model.area_median_income",
    "build_existing_model.heating_fuel",
]
x_labels = [
    "State",
    "Vintage",
    "Floor area",
    "Building type",
    "Census region", # include additional col for US
    "Federal poverty level",
    "Area median income",
    "Heating fuel type",
]

for gbc, xlab in zip(groupby_cols, x_labels):
    metric_cols = ["<100", "100", "101-199", "200", "201+"]
    dfi = dfm.groupby(gbc)[metric_cols].sum()
    dfc = dfm.groupby(gbc)["building_id"].count()
    
    dfi = dfi.divide(dfi.sum(axis=1), axis=0)
    dfi = sort_index(sort_index(dfi, axis=0), axis=1)
    dfc = sort_index(dfc.divide(dfc.sum()))
    
    if gbc == "build_existing_model.census_region":
        # add US total to data
        us = (dfm[metric_cols].sum() / dfm[metric_cols].sum().sum()).rename("US")
        dfi = pd.concat([us, dfi.T], axis=1).T
        dfi.index.name = gbc

        dfc = pd.concat([pd.Series(1, index=["US"]), dfc], axis=0)
        dfc.index.name = gbc

    if gbc == "build_existing_model.state":
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        fig.set_figwidth(18)
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="Panel capacity (A)")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"data__stacked_bar__{metric}.csv", index=True)
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="Panel capacity (A)")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"data__stacked_bar__{metric}.csv", index=True)

### [1.1] plot probability of labels by one variable - CA only

In [ ]:
if not has_filter:
    state = "CA" # <---
    cond = dfm["build_existing_model.state"]==state

    #########################################
    groupby_cols = [
        "build_existing_model.vintage",
        "build_existing_model.geometry_floor_area",
        "build_existing_model.geometry_building_type_recs",
        "build_existing_model.census_region", 
        "build_existing_model.federal_poverty_level",
        "build_existing_model.area_median_income",
    ]
    x_labels = [
        "Vintage",
        "Floor area",
        "Building type",
        "Census region",
        "Federal poverty level",
        "Area median income",
    ]

    for gbc, xlab in zip(groupby_cols, x_labels):
        metric_cols = ["<100", "100", "101-199", "200", "201+"]
        dfi = dfm.loc[cond].groupby(gbc)[metric_cols].sum()
        dfc = dfm.loc[cond].groupby(gbc)["building_id"].count()
        
        dfi = dfi.divide(dfi.sum(axis=1), axis=0)
        dfi = sort_index(sort_index(dfi, axis=0), axis=1)
        dfc = sort_index(dfc.divide(dfc.sum()))
        
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="Panel capacity (A)")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")

        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)

        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")

        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)

        fig.suptitle(f"{state} only")
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"CA_only__stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"CA_only__data__stacked_bar__{metric}.csv", index=True)

### [2] plot probability of labels by two variables

In [ ]:
### alternative

gbc = ["build_existing_model.geometry_floor_area", "build_existing_model.geometry_building_type_recs"]
xlab = "(Floor area, Building type)"

metric_cols = ["<100", "100", "101-199", "200", "201+"]
dfi = dfm.groupby(gbc)[metric_cols].sum()
dfc = dfm.groupby(gbc)["building_id"].count()

dfi = dfi.divide(dfi.sum(axis=1), axis=0)
dfc = sort_index(dfc.divide(dfc.sum()))
dfi = sort_index(sort_index(dfi, axis=0), axis=1)

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]}, figsize=(16,8))
dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="Panel capacity (A)")
ax1.set_ylim(0,1)
ax1.set_ylabel(f"Share of buildings")

for i, row in dfi.reset_index(drop=True).iterrows():
    for j in range(len(row)):
        ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)

dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
ax2.set_xlabel(xlab)
ax2.set_ylabel("Share of category")

for j in range(len(dfc)):
    ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)

plt.tight_layout()

metric = f"label_proba_by_{xlab}"
fig.savefig(output_dir / f"3stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")


### [2.1] plot probability of labels by one variable, subplot by Census Region

In [ ]:
group_bys = [
    "build_existing_model.vintage",
    "build_existing_model.geometry_floor_area",
    "build_existing_model.geometry_building_type_recs",
    "build_existing_model.federal_poverty_level",
    "build_existing_model.area_median_income",
]
labels = [
    "Vintage",
    "Floor area",
    "Building type",
    "Federal poverty level",
    "Area median income",
]

for gbc, xlab in zip(group_bys, labels):
    xlab = f"(Census Region, {xlab})"
    
    metric_cols = ["<100", "100", "101-199", "200", "201+"]
    dfi = dfm.groupby(["build_existing_model.census_region", gbc])[metric_cols].sum()
    dfc = dfm.groupby(["build_existing_model.census_region", gbc])["building_id"].count()
    
    dfi = dfi.divide(dfi.sum(axis=1), axis=0)
    dfc = sort_index(dfc.divide(dfc.sum()))
    dfi = sort_index(sort_index(dfi, axis=0), axis=1)
    
    fig, axes = plt.subplots(nrows=2, ncols=4, sharex=True, sharey="row", gridspec_kw={'height_ratios': [4,1]}, 
                             figsize=(3.75+1.25*dfm[gbc].nunique(), 6))
    for k, reg in enumerate(sorted(dfi.index.get_level_values(level=0).unique())):
        ax1 = axes[0,k]
        dfii = sort_index(sort_index(dfi.loc[(reg)], axis=0), axis=1)
        dfii.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1, legend=False)
        # ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="Panel capacity (A)")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
        ax1.set_title(reg)
        
        for i, row in dfii.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
                
        ax2 = axes[1,k]
        dfcc = sort_index(dfc.loc[(reg)])
        dfcc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2, legend=False)
        ax2.set_xticks(ticks = range(len(dfcc)), labels = format_labels(dfcc.index))
        ax2.set_xlabel(None)
        ax2.set_ylabel("Share of category")
        
        for j in range(len(dfcc)):
            ax2.text(j, dfcc.iloc[j]/2, f"{dfcc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)

    handles, labels = ax1.get_legend_handles_labels()
    fig.legend(reversed(handles), reversed(labels), title="Panel capacity (A)", loc='center left', bbox_to_anchor=(1, 0.5))
    
    fig.supxlabel(xlab)
    plt.tight_layout()
    
    metric = f"label_proba_by_{xlab}"
    fig.savefig(output_dir / f"2stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")

In [ ]:
group_bys = [
    "build_existing_model.vintage",
    "build_existing_model.geometry_floor_area",
    "build_existing_model.geometry_building_type_recs",
    "build_existing_model.federal_poverty_level",
    "build_existing_model.area_median_income",
]
labels = [
    "Vintage",
    "Floor area",
    "Building type",
    "Federal poverty level",
    "Area median income",
]

for group_by, xlab in zip(group_bys, labels):
    subplot_by = "build_existing_model.census_region" # fixed - for # of axes and fig output name
    metric_cols = ["<100", "100", "101-199", "200", "201+"]
    dfi = dfm.groupby([subplot_by, group_by])[metric_cols].sum()
    
    dfi = dfi.divide(dfi.sum(axis=1), axis=0)
    dfi = sort_index(sort_index(dfi, axis=0), axis=1)
    dfi = dfi.unstack(level=0)
    dfi = dfi.swaplevel(i=-2, j=-1, axis=1).sort_index(axis=1)
    
    # get count of subplot_by
    dfc = dfm.groupby(subplot_by)["building_id"].count()
    dfc = dfc/dfc.sum()
    
    fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(8,7))
    
    for k, reg in enumerate(dfi.columns.get_level_values(level=0).unique()):
        dfii = dfi.iloc[:, dfi.columns.get_level_values(level=0)==reg][reg]
        dfii = sort_index(sort_index(dfii, axis=0), axis=1)
    
        ax = axes[k //2,  k % 2]
        dfii.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax, legend=False)
        ax.set_ylim(0,1)
        ax.set_ylabel(f"Share of buildings")
        ax.set_xlabel(xlab)
        ax.set_xticks(ticks = range(len(dfii)), labels = format_labels(dfii.index))
        ax.set_title(f"{reg} (n={dfc[reg]*100:.0f}%)")
        
        for i, row in dfii.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(reversed(handles), reversed(labels), title="Panel capacity (A)", loc='center left', bbox_to_anchor=(1, 0.5))

    plt.tight_layout()

    metric = f"label_proba_by_{xlab}_subplot_by_census_region"
    fig.savefig(output_dir / f"1stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
    dfi.to_csv(output_dir / f"data__1stacked_bar__{metric}.csv", index=True)

### [3] plot rep value prediction vs. NEC

In [ ]:
def bin_panel_cap_headroom(df_column):
    # 10A single heating element heater
    # 5kVA ~ 20A (dryer)
    # 7.2kVA ~ 30A (EVSE)
    # 12kVA ~ 50A (cooking)

    try:
        df_column = df_column.astype(float)
    except ValueError as e:
        df_column = df_column.replace('', np.nan).astype(float)

    df_out = pd.Series(np.nan, index=df_column.index)
    df_out.loc[df_column<=0] = "0A"
    df_out.loc[(df_column>0) & (df_column<10)] = "1-9A"
    df_out.loc[(df_column>=10) & (df_column<20)] = "10-19A"
    df_out.loc[(df_column>=20) & (df_column<30)] = "20-29A"
    df_out.loc[(df_column>=30) & (df_column<50)] = "30-49A"
    df_out.loc[(df_column>=50)] = "50+A"

    categories = ["0A", "1-9A", "10-19A", "20-29A", "30-49A", "50+A"]
    df_out = pd.Categorical(df_out, ordered=True, categories=categories)

    assert df_out.isna().sum() == 0, f"{df_out.isna().sum()} NA values after binning panel capacity headroom"

    return df_out

In [ ]:
### combine prediction with NEC calc
print(f"Using {method} for representative value of panel amperage...")
df = dfn.join(
    dfm.set_index("building_id")[["predicted_panel_amp", "predicted_panel_amp_exact"]],
    on="building_id"
)
df["headroom_amp_220_83"] = df["predicted_panel_amp_exact"] - df["existing_amp_220_83"]
df["headroom_amp_220_87"] = df["predicted_panel_amp_exact"] - df["existing_amp_220_87"]
df["headroom_pct_220_83"] = (df["headroom_amp_220_83"] / df["predicted_panel_amp_exact"])*100
df["headroom_pct_220_87"] = (df["headroom_amp_220_87"] / df["predicted_panel_amp_exact"])*100

df["headroom_amp_bin_220_83"] = bin_panel_cap_headroom(df["headroom_amp_220_83"])
df["headroom_amp_bin_220_87"] = bin_panel_cap_headroom(df["headroom_amp_220_87"])
df

In [ ]:
# scatter plots
y_metrics = [
    "existing_amp_220_83",
    "existing_amp_220_87",
]

y_labels = [
    "Existing load (NEC 220.83) (A)",
    "Existing load (NEC 220.87) (A)",
]

for y_metric, y_label in zip(y_metrics, y_labels):
    x_metric = "predicted_panel_amp_exact"
    x_label = "Predicted panel capacity (A)"
    
    title = None
    dfp = df[[x_metric, y_metric, "predicted_panel_amp"]].dropna(how="any")
    assert dfp[x_metric].dtype == int, f"{x_metric=} is of type {dfp[x_metric].dtype=}"
    assert dfp[y_metric].dtype == float, f"{y_metric=} is of type {dfp[y_metric].dtype=}"
    
    fig, ax = plt.subplots()
    for bin, color in zip(
        ["<100", "100", "101-199", "200", "201+"],
        ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple'],
    ):
        x = dfp.loc[dfp["predicted_panel_amp"]==bin, x_metric]
        y = dfp.loc[dfp["predicted_panel_amp"]==bin, y_metric]
        ax.scatter(x, y, label=bin, c=color, alpha=0.8)
        
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_xlim(0, 1030)
    ax.set_ylim(-15, 715)
    ax.margins(x=0, y=0)
    
    lxy = np.array([
        min(ax.get_xlim()[0], ax.get_ylim()[0]), 
        max(ax.get_xlim()[1], ax.get_ylim()[1]), 
        ])
    ax.plot(lxy, lxy, label="axis of symmetry", ls="--", c="gray")
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # calculate % above and at or below x=y
    frac = dfp[y_metric]/dfp[x_metric]
    frac_above = len(frac[frac>1]) / len(frac)
    frac_at_below = 1-frac_above
    ax.text(0.05, 0.95, f"above line:\n{frac_above*100:.1f}%", ha="left", va="top", transform=ax.transAxes, 
            bbox=dict(facecolor='w', alpha=0.3))
    ax.text(0.95, 0.05, f"at/below line:\n{frac_at_below*100:.1f}%", ha="right", va="bottom", transform=ax.transAxes, 
            bbox=dict(facecolor='w', alpha=0.3))
    
    title_ext = f"(n = {len(dfp)})"
    if title is not None:
        title += f" {title_ext}"
    else:
        title = title_ext
    ax.set_title(title)
    if output_dir is not None:
        fig.savefig(output_dir / f"scatter__{y_metric}_by_{x_metric}_{method}.png", dpi=400, bbox_inches="tight")

In [ ]:
# box plots -- existing load
for panel_metric in [
    "existing_amp_220_83", 
    "existing_amp_220_87"
]:
    
    by_var = "predicted_panel_amp"
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
    df[[by_var, panel_metric]].boxplot(by=by_var, ax=ax1, showfliers=True)
    ax1.set_ylabel("Existing load (A)")
    ax1.set_ylim(-5,145) # adjust ylim by looking at natural ylim with showfliers=False in .boxplot()
    if "83" in panel_metric:
        ax1.set_title(f"Existing load per NEC 220.83 (n={len(df)})")
    else:
        ax1.set_title(f"Existing load per NEC 220.87 (n={len(df)})")
    
    dfc = sort_index(df[by_var].value_counts()) / len(df)
    pd.concat([
        pd.Series(0, index=[""]),
        dfc], axis=0).plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
    ax2.set_xlim(0.5,len(dfc)+0.5)
    ax2.set_xticks(ticks = range(1, len(dfc)+1), labels = format_labels(dfc.index), rotation=0)
    
    ax2.set_xlabel("Predicted panel bin (A)")
    ax2.set_ylabel("Share of category")
    
    for j in range(len(dfc)):
        ax2.text(j+1, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
    fig_file = f"box__{panel_metric}_by_{by_var}.png"
    if output_dir is not None:
        fig_file = output_dir / fig_file
    
    plt.suptitle(None)
    plt.tight_layout()
    fig.savefig(fig_file, dpi=400, bbox_inches="tight")

In [ ]:
panel_labels = ["<100", "100", "101-199", "200", "201+"]
existing_load_methods = ["existing_amp_220_83", "existing_amp_220_87"]

for mtd in existing_load_methods:
    print(f"\nUsing {mtd} to calculate existing load:")
    
    for label in panel_labels:        
        cond = df["predicted_panel_amp"].isin([label]) 
        
        loads = df.loc[cond, mtd]
        upper = np.percentile(loads, 75) + 1.5*(np.percentile(loads, 75) - np.percentile(loads, 25))
        lower = np.percentile(loads, 25) - 1.5*(np.percentile(loads, 75) - np.percentile(loads, 25))
        cond_outliers = cond & ((df[mtd]<lower) | (df[mtd]>upper))
        n_outliers = len(df.loc[cond_outliers])
        pct_outliers = n_outliers / len(loads) * 100
        
        print(f"For label: {label}...")
        print(f" - outliers: {n_outliers} ( {pct_outliers:.02f} % ) ")
        
        cond_err = cond & (df["predicted_panel_amp_exact"]<=df[mtd])
        n_err = len(df.loc[cond_err])
        pct_err = n_err / len(loads) * 100
        print(f" - homes where existing load >= panel amp: {n_err} ( {pct_err:.02f} % ) ")
        print(f" - homes where existing load < panel amp (with headroom): {len(loads) - n_err} ( {(100-pct_err):.02f} % ) ")

In [ ]:
# How much is existing load by NEC 220.87 lower than 220.83:
((df["existing_amp_220_87"]-df["existing_amp_220_83"])/df["existing_amp_220_83"]).describe()

In [ ]:
# How much is more headroom is calculated by NEC 220.87 than 220.83:
(df["headroom_amp_220_87"]-df["headroom_amp_220_83"]).describe()

In [ ]:
# error:= "predicted_panel_amp" > "existing_amp_220_87"
cond = df["predicted_panel_amp_exact"]<df["existing_amp_220_87"]
df_error = df.loc[cond]

print(len(df_error))
len(df_error) / len(df)

In [ ]:
# box plots -- Head room abs
for panel_metric in [
    "headroom_amp_220_83", 
    "headroom_amp_220_87"
]:
    
    by_var = "predicted_panel_amp"
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
    df[[by_var, panel_metric]].boxplot(by=by_var, ax=ax1, showfliers=True)
    ax1.set_ylabel("Headroom on panel (A)")
    ax1.set_ylim(-100, 650) # adjust ylim by looking at natural ylim with showfliers=False in .boxplot()
    if "83" in panel_metric:
        ax1.set_title(f"Available capacity per NEC 220.83 (n={len(df)})")
    else:
        ax1.set_title(f"Available capacity per NEC 220.87 (n={len(df)})")
    
    dfc = sort_index(df[by_var].value_counts()) / len(df)
    pd.concat([
        pd.Series(0, index=[""]),
        dfc], axis=0).plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
    ax2.set_xlim(0.5,len(dfc)+0.5)
    ax2.set_xticks(ticks = range(1, len(dfc)+1), labels = format_labels(dfc.index), rotation=0)
    
    ax2.set_xlabel("Predicted panel bin (A)")
    ax2.set_ylabel("Share of category")
    
    for j in range(len(dfc)):
        ax2.text(j+1, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
    fig_file = f"box__{panel_metric}_by_{by_var}.png"
    if output_dir is not None:
        fig_file = output_dir / fig_file
    
    plt.suptitle(None)
    plt.tight_layout()
    fig.savefig(fig_file, dpi=400, bbox_inches="tight")

In [ ]:
# tabulate fractions of homes by binned cap headroom
existing_load_methods = ["headroom_amp_bin_220_83", "headroom_amp_bin_220_87"]
for mtd in existing_load_methods:
    groupby_cols = ["predicted_panel_amp", mtd]
    dfi = df.groupby(groupby_cols)["building_id"].count().unstack().div(df.groupby(["predicted_panel_amp"])["building_id"].count(), axis=0)
    display(dfi)
    dfi.to_csv(output_dir / f"data__fraction_homes__{mtd}.csv", index=True)

In [ ]:
panel_labels = ["<100", "100", "101-199", "200", "201+"]
existing_load_methods = ["headroom_amp_220_83", "headroom_amp_220_87"]

for mtd in existing_load_methods:
    print(f"\nUsing {mtd} to calculate existing load:")

    table = []
    for label in panel_labels:        
        cond = df["predicted_panel_amp"].isin([label]) 
        
        loads = df.loc[cond, mtd]
        upper = np.percentile(loads, 75) + 1.5*(np.percentile(loads, 75) - np.percentile(loads, 25))
        lower = np.percentile(loads, 25) - 1.5*(np.percentile(loads, 75) - np.percentile(loads, 25))
        cond_outliers = cond & ((df[mtd]<lower) | (df[mtd]>upper))
        n_outliers = len(df.loc[cond_outliers])
        pct_outliers = n_outliers / len(loads) * 100
        
        print(f"For label: {label}...")
        print(f" - outliers: {n_outliers} ( {pct_outliers:.02f} % ) ")
    

In [ ]:
# box plots -- Head room %
for panel_metric in [
    "headroom_pct_220_83", 
    "headroom_pct_220_87"
]:
    
    by_var = "predicted_panel_amp"
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
    df[[by_var, panel_metric]].boxplot(by=by_var, ax=ax1, showfliers=True)
    ax1.set_ylabel("Relative headroom on panel (%)")
    ax1.set_ylim(-45, 110) # adjust ylim by looking at natural ylim with showfliers=False in .boxplot()
    if "83" in panel_metric:
        ax1.set_title(f"Available capacity per NEC 220.83 (n={len(df)})")
    else:
        ax1.set_title(f"Available capacity per NEC 220.87 (n={len(df)})")
    
    dfc = sort_index(df[by_var].value_counts()) / len(df)
    pd.concat([
        pd.Series(0, index=[""]),
        dfc], axis=0).plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
    ax2.set_xlim(0.5,len(dfc)+0.5)
    ax2.set_xticks(ticks = range(1, len(dfc)+1), labels = format_labels(dfc.index), rotation=0)
    
    ax2.set_xlabel("Predicted panel bin (A)")
    ax2.set_ylabel("Share of category")
    
    for j in range(len(dfc)):
        ax2.text(j+1, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
    fig_file = f"box__{panel_metric}_by_{by_var}.png"
    if output_dir is not None:
        fig_file = output_dir / fig_file
    
    plt.suptitle(None)
    plt.tight_layout()
    fig.savefig(fig_file, dpi=400, bbox_inches="tight")

### [4.1] State maps

In [ ]:
panel_label2 = "% dwelling units with 100 Amp or less"
title = "Electrical panels 100 Amp or less"
dfcond = df.loc[df["predicted_panel_amp"].isin(["<100", "100"])]
data2 = (dfcond.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label2)
data2.index.name = "State"
data2 = data2.reset_index()

# plot
fig = px.choropleth(data2, locations='State', color=panel_label2, title=title,
                    locationmode="USA-states", color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa")
fig.add_scattergeo(locations=data2['State'], text=data2[panel_label2].round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                  )

fig.show()
fig_file = f"map__{panel_label2}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
#fig.write_image(fig, fig_file, scale=6, width=800, height=500)
#data2.to_csv(f"{title}_state.csv")

In [ ]:
panel_label1 = "% dwelling units"
title =  "Electrical panels 200 Amp or more"
dfcond = df.loc[df["predicted_panel_amp"].isin(["<100", "100"])]
data1 = (dfcond.groupby(["build_existing_model.census_region"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.census_region"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label1)
data1.index.name = "census_region"
data1 = data1.reset_index()
data1.to_csv(f"{title}.csv")

In [ ]:
panel_label2 = "% dwelling units with 200 Amp or more"
title = "Electrical panels 200 Amp or more"
dfcond = df.loc[df["predicted_panel_amp"].isin(["200", "201+"])]
data2 = (dfcond.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label2)
data2.index.name = "State"
data2 = data2.reset_index()

# plot
fig = px.choropleth(data2, locations='State', color=panel_label2, title=title,
                    locationmode="USA-states", color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa")
fig.add_scattergeo(locations=data2['State'], text=data2[panel_label2].round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                  )

fig.show()
fig_file = f"map__{panel_label2}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
#fig.write_image(fig, fig_file, scale=6, width=800, height=500)
#data2.to_csv(f"{title}_state.csv")

In [ ]:
panel_label1 = "% dwelling units"
title =  "Electrical panels 200 Amp or more"
dfcond = df.loc[df["predicted_panel_amp"].isin(["200", "201+"])]
data1 = (dfcond.groupby(["build_existing_model.census_region"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.census_region"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label1)
data1.index.name = "census_region"
data1 = data1.reset_index()
data1.to_csv(f"{title}.csv")

In [ ]:
panel_label1 = "% dwelling units"
title = "30 Amp or more available capacity (NEC 220.83)"
dfcond = df.loc[df["headroom_amp_bin_220_83"].isin(["30-49A", "50+A"])]
data1 = (dfcond.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa")
fig.add_scattergeo(locations=data1['State'], text=data1[panel_label1].round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                  )

fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [ ]:
panel_label2 = "% dwelling units"
title = "30 Amp or more available capacity (NEC 220.87)"
dfcond = df.loc[df["headroom_amp_bin_220_87"].isin(["30-49A", "50+A"])]
data2 = (dfcond.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["build_existing_model.state"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label2)
data2.index.name = "State"
data2 = data2.reset_index()

# plot
fig = px.choropleth(data2, locations='State', color=panel_label2, title=title,
                    locationmode="USA-states", color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa")
fig.add_scattergeo(locations=data2['State'], text=data2[panel_label2].round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                  )

fig.show()
fig_file = f"map__{panel_label2}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

### [4.2] County maps

In [ ]:
# create county fips for plotting
df["county_fips"] = df["build_existing_model.county_and_puma"].apply(lambda x: x.split(",")[0])
df["county_fips"] = df["county_fips"].apply(lambda x: x[1:3]+x[4:7])


In [ ]:
# county level plot
panel_label1 = "% dwelling units"
title = "Electrical panels 100 Amp or less"
dfcond = df.loc[df["predicted_panel_amp"].isin(["<100", "100"])]
data1 = (dfcond.groupby(["county_fips"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["county_fips"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label1)
data1.index.name = "County"
data1 = data1.reset_index()

fig = px.choropleth(data1, geojson=counties, locations='County', color=panel_label1, title=title,
                           color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa",)

fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.show()
fig_file = f"map__county_{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file

#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [ ]:
panel_label2 = "% dwelling units"
title = "Electrical panels 200 Amp or more"
dfcond = df.loc[df["predicted_panel_amp"].isin(["200", "201+"])]
data2 = (dfcond.groupby(["county_fips"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["county_fips"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label2)
data2.index.name = "County"
data2 = data2.reset_index()

fig = px.choropleth(data2, geojson=counties, locations='County', color=panel_label2, title=title,
                           color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa",)

fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0, "l":0,"b":0}, height=500, width=800)
fig.show()
fig_file = f"map__county_{panel_label2}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file

#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [ ]:
panel_label1 = "% dwelling units"
title = "30 Amp or more available capacity (NEC 220.83)"
dfcond = df.loc[df["headroom_amp_bin_220_83"].isin(["30-49A", "50+A"])]
data1 = (dfcond.groupby(["county_fips"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["county_fips"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label1)
data1.index.name = "County"
data1 = data1.reset_index()

fig = px.choropleth(data1, geojson=counties, locations='County', color=panel_label1, title=title,
                           color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa",)

fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"l":0,"b":0})
fig.show()
fig_file = f"map__county_{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file

#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [ ]:
panel_label2 = "% dwelling units"
title = "30 Amp or more available capacity (NEC 220.87)"
dfcond = df.loc[df["headroom_amp_bin_220_87"].isin(["30-49A", "50+A"])]
data2 = (dfcond.groupby(["county_fips"])["build_existing_model.sample_weight"].sum() / \
        df.groupby(["county_fips"])["build_existing_model.sample_weight"].sum()*100).rename(panel_label2)
data2.index.name = "County"
data2 = data2.reset_index()

fig = px.choropleth(data2, geojson=counties, locations='County', color=panel_label2, title=title,
                           color_continuous_scale=px.colors.sequential.RdBu_r, scope="usa",)

fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0, "l":0,"b":0}, height=500, width=800)
fig.show()
fig_file = f"map__county_{panel_label2}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file

#pio.write_image(fig, fig_file, scale=6, width=800, height=500)

### Braker Space

In [1]:
import pandas as pd
df = pd.read_csv('test_data/euss1_2018_results_up00_clean__model_41138__tsv_based__predicted_panels_probablistically_assigned_braker_space.csv')

C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_20004\185135642.py:2: DtypeWarning: Columns (66,118,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('test_data/euss1_2018_results_up00_clean__model_41138__tsv_based__predicted_panels_probablistically_assigned_braker_space.csv')


In [ ]:
test = df.head(1000)
test.to_csv('test.csv')

In [ ]:
df["panel_slots_empty_bin"] = df["panel_slots_empty"]
df.loc[(df["panel_slots_empty"] >= 5) & (df["panel_slots_empty"] <10), 'panel_slots_empty_bin'] = "5-10"
df.loc[df["panel_slots_empty"] >= 10, 'panel_slots_empty_bin'] = "10+"

In [ ]:
df["major_elec_load_count"].unique()

In [ ]:
df.loc[df["major_elec_load_count"] == 0, 'ml_0'] = 1
df.loc[df["major_elec_load_count"] == 1, 'ml_1'] = 1
df.loc[df["major_elec_load_count"] == 2, 'ml_2'] = 1
df.loc[df["major_elec_load_count"] == 3, 'ml_3'] = 1
df.loc[df["major_elec_load_count"] == 4, 'ml_4'] = 1
df.loc[df["major_elec_load_count"] == 5, 'ml_5'] = 1
df['ml_0'].fillna(0, inplace=True)
df['ml_1'].fillna(0, inplace=True)
df['ml_2'].fillna(0, inplace=True)
df['ml_3'].fillna(0, inplace=True)
df['ml_4'].fillna(0, inplace=True)
df['ml_5'].fillna(0, inplace=True)

In [ ]:
df.loc[df["panel_slots_empty_bin"] == 0, '0'] = 1
df.loc[df["panel_slots_empty_bin"] == 1, '1'] = 1
df.loc[df["panel_slots_empty_bin"] == 2, '2'] = 1
df.loc[df["panel_slots_empty_bin"] == 3, '3'] = 1
df.loc[df["panel_slots_empty_bin"] == 4, '4'] = 1
df.loc[df["panel_slots_empty_bin"] == '5-10', '5-10'] = 1
df.loc[df["panel_slots_empty_bin"] == '10+', '10+'] = 1
df['0'].fillna(0, inplace=True)
df['1'].fillna(0, inplace=True)
df['2'].fillna(0, inplace=True)
df['3'].fillna(0, inplace=True)
df['4'].fillna(0, inplace=True)
df['5-10'].fillna(0, inplace=True)
df['10+'].fillna(0, inplace=True)
df = df.loc[df["build_existing_model.vacancy_status"]=="Occupied"].reset_index(drop=True)

In [ ]:
dfm = df
groupby_cols = [
    "build_existing_model.state",
    "build_existing_model.vintage",
    "build_existing_model.geometry_floor_area",
    "build_existing_model.geometry_building_type_recs",
    "build_existing_model.census_region", 
    "build_existing_model.federal_poverty_level",
    "build_existing_model.area_median_income",
    "build_existing_model.heating_fuel",
    "predicted_panel_amp",
]
x_labels = [
    "State",
    "Vintage",
    "Floor area",
    "Building type",
    "Census region", # include additional col for US
    "Federal poverty level",
    "Area median income",
    "Heating fuel type",
    "predicted panel amp",
]

for gbc, xlab in zip(groupby_cols, x_labels):
    metric_cols = ['0','1','2','3','4','5-10','10+']
    dfi = dfm.groupby(gbc)[metric_cols].sum()
    dfc = dfm.groupby(gbc)["building_id"].count()
    
    dfi = dfi.divide(dfi.sum(axis=1), axis=0)
    dfi = sort_index(sort_index(dfi, axis=0), axis=1)
    dfc = sort_index(dfc.divide(dfc.sum()))
    
    if gbc == "build_existing_model.census_region":
        # add US total to data
        us = (dfm[metric_cols].sum() / dfm[metric_cols].sum().sum()).rename("US")
        dfi = pd.concat([us, dfi.T], axis=1).T
        dfi.index.name = gbc

        dfc = pd.concat([pd.Series(1, index=["US"]), dfc], axis=0)
        dfc.index.name = gbc

    if gbc == "build_existing_model.state":
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        fig.set_figwidth(18)
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="panel_slots_empty_bin")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"braker_space/stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"braker_space/data__stacked_bar__{metric}.csv", index=True)
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="panel_slots_empty_bin")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"braker_space/stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"braker_space/data__stacked_bar__{metric}.csv", index=True)

In [ ]:
dfm = df
groupby_cols = [
    "build_existing_model.state",
    "build_existing_model.vintage",
    "build_existing_model.geometry_floor_area",
    "build_existing_model.geometry_building_type_recs",
    "build_existing_model.census_region", 
    "build_existing_model.federal_poverty_level",
    "build_existing_model.area_median_income",
    "build_existing_model.heating_fuel",
    "predicted_panel_amp",
]
x_labels = [
    "State",
    "Vintage",
    "Floor area",
    "Building type",
    "Census region", # include additional col for US
    "Federal poverty level",
    "Area median income",
    "Heating fuel type",
    "predicted panel amp",
]

for gbc, xlab in zip(groupby_cols, x_labels):
    metric_cols = ['ml_0','ml_1', 'ml_2', 'ml_3','ml_4','ml_5']
    dfi = dfm.groupby(gbc)[metric_cols].sum()
    dfc = dfm.groupby(gbc)["building_id"].count()
    
    dfi = dfi.divide(dfi.sum(axis=1), axis=0)
    dfi = sort_index(sort_index(dfi, axis=0), axis=1)
    dfc = sort_index(dfc.divide(dfc.sum()))
    
    if gbc == "build_existing_model.census_region":
        # add US total to data
        us = (dfm[metric_cols].sum() / dfm[metric_cols].sum().sum()).rename("US")
        dfi = pd.concat([us, dfi.T], axis=1).T
        dfi.index.name = gbc

        dfc = pd.concat([pd.Series(1, index=["US"]), dfc], axis=0)
        dfc.index.name = gbc

    if gbc == "build_existing_model.state":
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        fig.set_figwidth(18)
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="major_elec_load_count")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=7)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"braker_space/stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"braker_space/data__stacked_bar__{metric}.csv", index=True)
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [4,1]})
        dfi.plot(kind="bar", stacked=True,  alpha=0.8, width=0.7, ax=ax1)
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5), reverse=True, title="major_elec_load_count")
        ax1.set_ylim(0,1)
        ax1.set_ylabel(f"Share of buildings")
    
        for i, row in dfi.reset_index(drop=True).iterrows():
            for j in range(len(row)):
                ax1.text(i, row.iloc[0:j].sum()+row.iloc[j]/2, f"{row.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        dfc.plot(kind="bar", stacked=False, color="gray", alpha=0.8, width=0.7, ax=ax2)
        ax2.set_xticks(ticks = range(len(dfc)), labels = format_labels(dfc.index))
        ax2.set_xlabel(xlab)
        ax2.set_ylabel("Share of category")
    
        for j in range(len(dfc)):
            ax2.text(j, dfc.iloc[j]/2, f"{dfc.iloc[j]*100:.0f}%", ha="center", va="center", fontsize=8)
    
        plt.tight_layout()
        
        metric = f"label_proba_by_{xlab}"
        fig.savefig(output_dir / f"major_elec_load_count/stacked_bar__{metric}.png", dpi=400, bbox_inches="tight")
        dfi.to_csv(output_dir / f"major_elec_load_count/data__stacked_bar__{metric}.csv", index=True)

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="panel_slots_empty", color="predicted_panel_amp",
                   marginal="box", # or violin, rug
                   hover_data=df.columns)
fig.show()

In [ ]:
fig.write_image(output_dir / f"braker_space/stacked_bar__{metric}.png") 

### Miscellaneous Script

In [ ]:
### investigate outliers in full survey dataset
data = pd.read_excel("/Uses/lliu2/Documents/GitHub/resstock2/Electrical_Panels_EUSS/model_20240122/outlier_summary.xlsx", 
                   sheet_name="floor area", header=1)
data 

In [ ]:
fab = data["geometry_floor_area_bin"].unique()

res = []
for bin in fab:
    dat = data.loc[data["geometry_floor_area_bin"]==bin,["panel_amp_pre", "total count"]]
    panel_list = np.repeat(dat["panel_amp_pre"], dat["total count"]).values
    
    # calculate IQR
    P75, P25 = np.percentile(panel_list, 75), np.percentile(panel_list, 25)
    LB = P25 - 1.5*(P75-P25)
    UB = P75 + 1.5*(P75-P25)
    LB_outliers = panel_list[panel_list<LB]
    UB_outliers = panel_list[panel_list>UB]

    res.append(
        pd.Series([bin, len(panel_list), LB, UB, len(LB_outliers), len(UB_outliers), LB_outliers, UB_outliers],
                  index = ["floor_area", "n_samples", "lower_bound", "upper_bound", "n_lb_outliers", "n_ub_outliers", "lb_outliers", "ub_outliers"]
                  )
    )
res = pd.concat(res, axis=1).transpose()

res